### HOSPITAIS

#### Libs

In [3]:
import pandas as pd
import plotly.graph_objects as go

DATADIR = '../app/datasets/inputs'
DATAOUT = '../app/datasets/outputs'

#### Base Hospitais

In [4]:
hosp_file_columns = {
    'Name,C,254': 'estabelecimento',
    'MUNICIPIO,C,254': 'municipio', 
    'PARTOS,C,254': 'proced_partos', 
    'GESTAO,C,254': 'gestao', 
    'LAT,N,11,10': 'lat', 
    'LONG,N,11,10': 'lng'
}

df_hosp = pd.read_csv(
    f'{DATADIR}/HOSPITAL_ESTADUAL_PARTO_2.csv', 
    sep=';',
    usecols=hosp_file_columns.keys(),
    # dtype={'LAT,N,11,10': str, 'LONG,N,11,10': 'str'}
).\
    rename(columns=hosp_file_columns)

df_hosp['lat'] = df_hosp['lat'].str.replace(',', '.').astype('float32')
df_hosp['lng'] = df_hosp['lng'].str.replace(',', '.').astype('float32')

df_hosp.head()

,estabelecimento,municipio,proced_partos,gestao,lat,lng
0,HOSPITAL AGAMENON MAGALHAES,RECIFE,SIM,ESTADUAL,-8.030304,-34.907127
1,HOSPITAL BARAO DE LUCENA,RECIFE,SIM,ESTADUAL,-8.038482,-34.938984
2,HOSPITAL COLONIA PROFESSOR ALCIDES CODECEIRA (...,IGARASSU,NAO,ESTADUAL,-7.867347,-34.910435
3,HOSPITAL CORREIA PICANCO,RECIFE,NAO,ESTADUAL,-8.033935,-34.904743
4,HOSPITAL DA RESTAURACAO,RECIFE,NAO,ESTADUAL,-8.053505,-34.897739


#### Base Leitos

In [6]:
leitos_file_columns = {
    'cd_nome_estab': 'cd_estab',
    'nome_estab': 'estabelecimento', 
    'OBSTETRICIA CIRURGICA': 'obst_cirurg', 
    'OBSTETRICIA CLINICA': 'obst_clinica', 
    'OBSTETRICIA CLINICA': 'obst_clinica', 
    'UTI NEONATAL - TIPO I': 'uti_neo_i', 
    'UTI NEONATAL - TIPO II': 'uti_neo_ii', 
    'UTI NEONATAL - TIPO III': 'uti_neo_iii', 
    'UNIDADE DE CUIDADOS INTERMED NEONATAL CONVENCIONAL': 'uci_neo_conv', 
    'UNIDADE DE CUIDADOS INTERMED NEONATAL CANGURU': 'uci_neo_canguru', 
    # 'total_leitos_obstetricos': 'total_leitos_obstetricos', 
}

df_leitos_obst = pd.read_excel(
    # f'{DATADIR}/leitos_obstetricos.csv', 
    f'{DATADIR}/hospitais_por_leito.xlsx', 
    usecols=leitos_file_columns.keys(),
    # dtype={'LAT,N,11,10': str, 'LONG,N,11,10': 'str'}
).\
    rename(columns=leitos_file_columns)

# df_leitos_obst['cd_estab'] = df_leitos_obst['cd_nm_estabelecimento'].apply(lambda x: x[:7])


df_leitos_obst.head()

,cd_estab,estabelecimento,obst_cirurg,obst_clinica,uti_neo_i,uti_neo_ii,uti_neo_iii,uci_neo_conv,uci_neo_canguru
0,0000019,POLICLINICA DR JAMACI DE MEDEIROS,0,0,0,0,0,0,0
1,0000027,CASA DE SAUDE SANTA HELENA,2,6,0,0,0,0,0
2,0000035,HOSPITAL MENDO SAMPAIO,0,0,0,0,0,0,0
3,0000043,POLICLINICA DR MANUEL GOMES,0,0,0,0,0,0,0
4,0000051,POLICLINICA VICENTE MENDES,0,0,0,0,0,0,0


#### Join: Leitos + Hospitais Estaduais

In [7]:
df_hosp_leitos = df_hosp.merge(df_leitos_obst, how='left', on='estabelecimento')

df_hosp_leitos.cd_estab.isna().sum() / len(df_hosp_leitos.cd_estab)

0.12244897959183673

Como 38% dos hospitais ainda sem código, vamos utilizar outra base...

#### Novo Join

In [8]:
df_estab = pd.read_csv(f'{DATADIR}/estabelecimentos_estaduais.csv', sep=';', usecols=['Estabel-CNES PE'])
df_estab['cd_estab'] = df_estab['Estabel-CNES PE'].apply(lambda x: x[:7])
df_estab['estabelecimento'] = df_estab['Estabel-CNES PE'].apply(lambda x: x[8:])

df_estab.head()


,Estabel-CNES PE,cd_estab,estabelecimento
0,0000396 HOSPITAL DAS CLINICAS,0000396,HOSPITAL DAS CLINICAS
1,0000418 HOSPITAL AGAMENON MAGALHAES,0000418,HOSPITAL AGAMENON MAGALHAES
2,0000426 HOSPITAL OTAVIO DE FREITAS,0000426,HOSPITAL OTAVIO DE FREITAS
3,0000434 IMIP,0000434,IMIP
4,0000477 HOSPITAL OSWALDO CRUZ,0000477,HOSPITAL OSWALDO CRUZ


In [11]:
cols_ = ['obst_cirurg','obst_clinica', 'uti_neo_i', 'uti_neo_ii', 'uti_neo_iii', 'uci_neo_conv', 'uci_neo_canguru']
df_hosp_leitos2 = df_hosp_leitos.drop('cd_estab', axis=1).merge(df_estab, how='left', on='estabelecimento')
df_hosp_leitos2[cols_] = df_hosp_leitos2[cols_].fillna(0).astype(int)
df_hosp_leitos2.head(2)

,estabelecimento,municipio,proced_partos,gestao,lat,lng,obst_cirurg,obst_clinica,uti_neo_i,uti_neo_ii,uti_neo_iii,uci_neo_conv,uci_neo_canguru,Estabel-CNES PE,cd_estab
0,HOSPITAL AGAMENON MAGALHAES,RECIFE,SIM,ESTADUAL,-8.030304,-34.907127,62,27,0,15,0,15,8,0000418 HOSPITAL AGAMENON MAGALHAES,0000418
1,HOSPITAL BARAO DE LUCENA,RECIFE,SIM,ESTADUAL,-8.038482,-34.938984,61,28,28,18,0,16,10,2427427 HOSPITAL BARAO DE LUCENA,2427427


In [12]:
df_hosp_leitos2.cd_estab.isna().sum() / len(df_hosp_leitos2.cd_estab)

0.12244897959183673

#### Mapa

In [20]:
df_hosp_leitos_com_obst = df_hosp_leitos2.query("proced_partos == 'SIM'")
df_hosp_leitos_sem_obst = df_hosp_leitos2.query("proced_partos == 'NAO'")

def hover_template(df):
    cols_hover = ['estabelecimento', 'municipio']  + cols_
    hover_text = [
        '<b>{}</b><br>{}<br><br><b>Leitos Obstetrícia</b><br>Cirúgicos: {} / Clínica: {}<br><br><b>UTI Neo</b><br>Tipo I: {} / Tipo II: {} / Tipo III: {}<br><br><b>UCI Neo</b><br>convencional: {} / Caguru: {}'.format(
            txt[0], txt[1], txt[2], txt[3], txt[4], txt[5], txt[6], txt[7], txt[8]
            ) 
        for txt in df[cols_hover].values
        ]
    return hover_text

fig = go.Figure([
    go.Scattermapbox(
        name='COM leitos obst.',
        lat=df_hosp_leitos_com_obst['lat'],
        lon=df_hosp_leitos_com_obst['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=13
            # size=df_hosp_leitos_com_obst['total_leitos_obstetricos'] / 2.5
        ),
        hovertext=hover_template(df_hosp_leitos_com_obst),
        hovertemplate="%{hovertext}"
    ),
    go.Scattermapbox(
        name='SEM leitos obst.',
        lat=df_hosp_leitos_sem_obst['lat'],
        lon=df_hosp_leitos_sem_obst['lng'],
        mode='markers',
        marker=dict(
            size=14
        ),
        hovertext=hover_template(df_hosp_leitos_sem_obst),
        hovertemplate="%{hovertext}"
    ),
])

fig.update_layout(
    title="Estabelecimentos/Hospitais",
    mapbox_style="carto-positron",
    mapbox_zoom=6, 
    mapbox_center = {"lat": -8.5517, "lon": -37.7073}
)

fig.show()

#### Exportando

In [21]:
df_hosp_leitos2.to_parquet(f'{DATAOUT}/estabelecimentos.parquet.gzip')